# Part 1

I solved several challenges with dense matrices, so today I'm making a dense matrix class.

In [1]:
import itertools

class Matrix:
    def __init__(self, rows, cols, initial=None):
        self._rows = rows
        self._cols = cols
        self._cells = [[initial] * cols for _ in range(rows)]
    
    def __repr__(self):
        return 'Matrix<{}x{}>'.format(self.rows, self.cols)
    
    def __str__(self):
        render = ''
        for row in self._cells:
            render += ''.join('·' if v is None else str(v) for v in row) + '\n'
        return render

    def __getitem__(self, key):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        return self._cells[row][col]
    
    def __setitem__(self, key, val):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        self._cells[row][col] = val
    
    def clone(self):
        m = Matrix(self._rows, self._cols)
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            m[row,col] = self[row,col]
        return m

    def get(self, row, col, default=None):
        try:
            return self[row,col]
        except (IndexError,KeyError):
            return default
    
    @property
    def size(self):
        return self._rows, self._cols

    def values(self):
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            yield self[row, col]

In [2]:
m = Matrix(3, 5)
print(m)
print(m.size)

·····
·····
·····

(3, 5)


In [3]:
m[0,0] = 'A'
m[1,1] = 'B'
m2 = m.clone()
m[2,4] = 'C'
print(m)
print(m2)

A····
·B···
····C

A····
·B···
·····



In [4]:
print(list(m2.values()))

['A', None, None, None, None, None, 'B', None, None, None, None, None, None, None, None]


My biggest question is how big the input will be. How many coordinates? How big of a matrix?

In [5]:
# I've flipped each coordinate, because I like the (row,column) coordinates.
coordinates = list()
with open('input.txt') as input_:
    for line in input_:
        col, row = tuple(map(int, line.split(',')))
        coordinates.append((row, col))
print(coordinates[:5])
print(len(coordinates))

[(292, 152), (90, 163), (65, 258), (147, 123), (42, 342)]
50


In [6]:
print('min row', min(c[0] for c in coordinates))
print('max row', max(c[0] for c in coordinates))
print('min col', min(c[1] for c in coordinates))
print('max col', max(c[1] for c in coordinates))

min row 42
max row 356
min col 40
max col 359


Ok so I think I should take each cell and compute its manhattan distance to each coordinate. This seems pretty expensive: R * C * len(coordinates) but maybe it could work.

In [7]:
import collections
import heapq
import string

names = string.ascii_uppercase + string.ascii_lowercase

def manhattan(mat, coordinates):
    ''' Label each cell with its closest coordinate. '''
    rows, cols = mat.size
    for mrow, mcol in itertools.product(range(0, rows), range(0, cols)):
        # Create a heap, where each value is a tuple (dist, name)
        distances = list()
        for i, (crow, ccol) in enumerate(coordinates):
            name = names[i]
            dist = abs(mrow-crow) + abs(mcol-ccol)
            heapq.heappush(distances, (dist, name))
        first = heapq.heappop(distances)
        second = heapq.heappop(distances)
        # We only save a value if the first and second values are different, i.e.
        # one of the coordinates is closer than the other.
        if first[0] != second[0]:
            mat[mrow, mcol] = first[1]

In [8]:
# I've flipped each coordinate, because I like the (row,column) coordinates.
test = [
    (1, 1),
    (6, 1),
    (3, 8),
    (4, 3),
    (5, 5),
    (9, 8),
]
m = Matrix(11, 10)
for i, c in enumerate(test):
    m[c] = i
print(m)

··········
·0········
··········
········2·
···3······
·····4····
·1········
··········
··········
········5·
··········



In [9]:
%%time
manhattan(m, test)

CPU times: user 324 µs, sys: 44 µs, total: 368 µs
Wall time: 370 µs


In [10]:
print(m)

AAAAA·CCCC
AAAAA·CCCC
AAADDECCCC
AADDDECCCC
··DDDEECCC
BB·DEEEECC
BBB·EEEE··
BBB·EEEFFF
BBB·EEFFFF
BBB·FFFFFF
BBB·FFFFFF



Great, this matches the test case! Now let's try it on the full data set.

In [11]:
%%time
m = Matrix(400, 400)
manhattan(m, coordinates)

CPU times: user 2.87 s, sys: 0 ns, total: 2.87 s
Wall time: 2.87 s


In [ ]:
# helpful for debugging but takes up a lot of space:
# print(m)

In [12]:
# Now lets count how many of each letter there are in the matrix
name_count = collections.Counter(m.values())
print(name_count.most_common(3))

[('Z', 11095), ('i', 8627), ('c', 7507)]


I can verify the name counts by printing the matrix, putting into a text editor, and doing a case-sensitive search for the letter Z. I get the same result: 11,095.


In [16]:
# Now we need to remove all letters that touch the edges.
def get_edge_names(mat):
    ''' Return set of names of all areas that touch the edge of the matrix. '''
    edge_names = set()
    rows, cols = mat.size
    for row in range(rows):
        edge_names.add(mat[row,0])
        edge_names.add(mat[row,cols-1])
    for col in range(cols):
        edge_names.add(mat[0,col])
        edge_names.add(mat[rows-1,col])
    return edge_names

In [18]:
edge_names = get_edge_names(m)
print(len(edge_names))
print(edge_names)

25
{'o', 'Z', 'F', 'r', 'G', 'h', 'q', 'K', 'H', 'j', 'M', None, 'W', 'Q', 'f', 'C', 'S', 't', 'c', 'P', 's', 'x', 'i', 'n', 'E'}


In [19]:
print(len(name_count))
for edge_name in edge_names:
    name_count.pop(edge_name)
print(len(name_count))

51
26


In [20]:
# Now the count of most common name should be the solution:
name_count.most_common(1)

[('V', 3687)]

# Part 2

In [21]:
def manhattan2(mat, coordinates, threshold):
    ''' Label cells that have cumulative distance < threshold. '''
    rows, cols = mat.size
    for mrow, mcol in itertools.product(range(0, rows), range(0, cols)):
        distance = 0
        for i, (crow, ccol) in enumerate(coordinates):
            distance += abs(mrow-crow) + abs(mcol-ccol)
        if distance < threshold:
            mat[mrow, mcol] = '#'

In [22]:
# I've flipped each coordinate, because I like the (row,column) coordinates.
test = [
    (1, 1),
    (6, 1),
    (3, 8),
    (4, 3),
    (5, 5),
    (9, 8),
]
m = Matrix(11, 10)
for i, c in enumerate(test):
    m[c] = i
print(m)

··········
·0········
··········
········2·
···3······
·····4····
·1········
··········
··········
········5·
··········



In [23]:
%%time
manhattan2(m, test, 32)
print(m)

··········
·0········
··········
···###··2·
··#####···
··#####···
·1·###····
··········
··········
········5·
··········

CPU times: user 429 µs, sys: 11 µs, total: 440 µs
Wall time: 423 µs


This matches the test case, so let's move onto full data set.

In [24]:
%%time
m = Matrix(400, 400)
manhattan2(m, coordinates, 10_000)

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.34 s


In [ ]:
# helpful for debugging but takes up a lot of space:
# print(m)

In [26]:
# Count the number of '#' characters
hash_count = collections.Counter(m.values())
hash_count.pop(None)
print(hash_count.most_common(1))

[('#', 40134)]
